## 1. Import Libraries

In [1]:
# Feature_Engineering
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import *
from pyspark.sql.functions import col, count, sum
# Modeling
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import linear_model, datasets
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score
import numpy as np
from pyspark.ml.feature import OneHotEncoder

## 2. Load Data and Convert to Spark Data Frame

In [2]:
# read files
sc.textFile("loan.csv").take(1)

[u'id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m']

In [3]:
# load data as dataframe
loan_df=spark.read.csv("loan.csv",header=True)

In [4]:
loan_df.rdd.getNumPartitions()
type(loan_df)

pyspark.sql.dataframe.DataFrame

## 3. Create response variable and features
### 3.1 Remove some columns based on EDA results

In [5]:
# loan_df1 = loan_df.drop('desc','mths_since_last_delinq','mths_since_last_record','next_pymnt_d',
#                         'mths_since_last_major_derog','annual_inc_joint','dti_joint','verification_status_joint',
#                         'open_acc_6m','open_il_6m','open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il',
#                         'il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','inq_fi','total_cu_tl',
#                         'inq_last_12m', # with a lot NA
#                         'id','member_id','collection_recovery_fee','last_pymnt_amnt','last_pymnt_d','out_prncp','out_prncp_inv',
#                         'pymnt_plan','recoveries','term','title','total_pymnt','total_pymnt_inv','total_rec_int',
#                         'total_rec_late_fee','total_rec_prncp','url','verification_status', 'initial_list_status', 
#                         'last_credit_pull_d','policy_code','emp_title','last_credit_pull_d' # domain knowledge
#                        )

#is that intentionally not leaving out drops?

In [6]:
# print loan_df1.head(1)
# print len(loan_df1.columns)

In [148]:
loan_df2=loan_df.select(
    loan_df.loan_amnt.cast("float"),
    loan_df.funded_amnt.cast("float"),
    loan_df.funded_amnt_inv.cast("float"),
    loan_df.int_rate.cast("float"),  #convert to float
    loan_df.installment.cast("float"), #convert to float
    'grade',
    'sub_grade',   # NEED TO BE DUMMIED
    'emp_length',
    'home_ownership', #group & make it dummy, done 
    loan_df.annual_inc.cast("float"),
    # 'issue_d',  #why comment out?
    'loan_status', 
    # response variable (pending for change after group discussion)
    'purpose',  #make it dummy
#     'zip_code', #interesting, worth dummying, might run into problems tho
#     'addr_state', #dummy
    'verification_status',
#     'initial_list_status',
    loan_df.dti.cast("float"),#float done
    loan_df.delinq_2yrs.cast("integer"),  
    #take out NAs, 2? 1, 3, mostly 0
    # 'earliest_cr_line',
    loan_df.inq_last_6mths.cast("integer"), #similar to delingq_2yr
    loan_df.open_acc.cast("integer"), 
    loan_df.pub_rec.cast("integer"), #what is it? 0,1
    loan_df.revol_bal.cast("float"), #done
    loan_df.revol_util.cast("float"),#done
    loan_df.total_acc.cast("integer"),
    #loan_df.last_credit_pull_d.cast("integer"), #date
    loan_df.acc_now_delinq.cast("integer"),#ok
    loan_df.tot_coll_amt.cast("float"), #done
    loan_df.tot_cur_bal.cast("float"), #done
    loan_df.total_rev_hi_lim.cast("integer") #what is this
)

### 3.2 Create response variable and remove rows with no valid response variable 

In [149]:
# needs to fix that, encoding doesn't seem to be right

def whetherpaid(x):
    if x in ['Default', 'Charged Off', 'Does not meet the credit policy. Status:Charged Off']:
        return 1
    elif x in ['Does not meet the credit policy. Status:Fully Paid', 'Fully Paid']:
        return 0
    else:
        return -1

In [150]:
paidflag = udf(lambda x: whetherpaid(x))

In [151]:
#subject to change
loan_df3 = loan_df2.withColumn(
    'paid_flag',
    paidflag('loan_status').cast("integer")
        ).where("paid_flag != -1").drop('loan_status')

In [152]:
loan_df3.printSchema()


root
 |-- loan_amnt: float (nullable = true)
 |-- funded_amnt: float (nullable = true)
 |-- funded_amnt_inv: float (nullable = true)
 |-- int_rate: float (nullable = true)
 |-- installment: float (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- purpose: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- acc_now_delinq: integer (nullable = true)
 |-- tot_coll_amt: float (nullable = true)
 |-- tot_cur_bal: float (nullable = true)
 |-- total_rev_hi_lim: integ

### 3.3 Add more features
#### 3.3.1 Create a numeric feature for "emp_length"

In [181]:
import re
def convert_to_int(s):
    s = re.sub('\\D', '', s)  #remove any non-digital character
    #\d matches any digital, #\D matches any non-digital
    try:
        return s
    except ValueError:
        return 'NaN'

emp_to_num = udf(convert_to_int)
loan_df4 = loan_df3.withColumn('emp_len',emp_to_num('emp_length').cast('integer')).drop('emp_length')

#### 3.3.2 Create numeric variable for grade 

In [182]:
# count how many nulls in each column
def count_null(c):
    return sum(col(c).isNull().cast("integer")).alias(c)

exprs = [count_null(c) for c in loan_df4.columns[0:9]]
loan_df4.agg(*exprs).show()

+---------+-----------+---------------+--------+-----------+-----+---------+--------------+----------+
|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|grade|sub_grade|home_ownership|annual_inc|
+---------+-----------+---------------+--------+-----------+-----+---------+--------------+----------+
|        0|          0|              0|       0|          0|    0|        0|             0|         4|
+---------+-----------+---------------+--------+-----------+-----+---------+--------------+----------+



#### 3.3.3 Create numeric variable for grade 

In [183]:
def home_ownership_func(x):
    if x in ['ANY','OTHER','NONE']:
        return 'Other'
    else: 
        return x

home_ownership = udf(home_ownership_func)  #fixed a function
loan_df5 = loan_df4.withColumn('home_ownership',home_ownership('home_ownership'))

In [184]:
loan_df5.select('home_ownership').groupBy('home_ownership').count().show()

+--------------+------+
|home_ownership| count|
+--------------+------+
|           OWN| 22282|
|         Other|   228|
|          RENT|107831|
|      MORTGAGE|126598|
+--------------+------+



#### 3.3.4 Add loan_inc_ratio feature 

In [185]:
def calculate_ratio(a, b):
    try:
        return a/float(b)
    except TypeError:
        return None
    except ZeroDivisionError:
        return None

ratio = udf(calculate_ratio) #define a udf_function ratio

In [186]:
loan_df6 = loan_df5.withColumn(
    'loan_inc_ratio',ratio('loan_amnt','annual_inc'
                          ).cast('float'))

In [187]:
loan_df7 = loan_df6.withColumn(
    'instal_inc_ratio',
    ratio('installment','annual_inc').cast('float'))

#### 3.3.5 Add feature instal_inc_ratio


In [188]:
def calculate_monthly_ratio(a, b):
    try:
        return a/(float(b)/12)
    except TypeError:
        return None
    except ZeroDivisionError:
        return None
    
monthly_ratio = udf(calculate_monthly_ratio)

In [189]:
loan_df8 = loan_df7.withColumn(
    'instal_inc_ratio',
    ratio('installment','annual_inc').cast('float'))

#### 3.3.6 Create dummy variables
* Use StringIndexer to encode a string column of labels to a column of label indices, and most frequent label gets index 0.

* Use OneHotEncoder to convert indices into dummy vectors

In [190]:
convert_list = [      
               'purpose',
               'grade',
               'sub_grade',
               'verification_status',
               'home_ownership',
                ]

for item in convert_list:
    print item
    indexer = StringIndexer(inputCol=item, outputCol=item + 'Index')    
    loan_df8 = indexer.fit(loan_df8).transform(loan_df8).drop(item)
    onehotenc = OneHotEncoder(inputCol=item + 'Index', outputCol=item+"-onehot", dropLast=False)
    # we can experiment with True
    loan_df8 = onehotenc.transform(loan_df8).drop(item+'Index')
    print item

purpose
purpose
grade
grade
sub_grade
sub_grade
verification_status
verification_status
home_ownership
home_ownership


In [191]:
loan_df8 = loan_df8.fillna(0.0, ['tot_coll_amt','tot_cur_bal', 'total_rev_hi_lim'])
loan_df9 = loan_df8.dropna()

In [192]:
loan_df9.columns

['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'paid_flag',
 'emp_len',
 'loan_inc_ratio',
 'instal_inc_ratio',
 'purpose-onehot',
 'grade-onehot',
 'sub_grade-onehot',
 'verification_status-onehot',
 'home_ownership-onehot']

In [193]:
loan_df_final = loan_df9.select(
 'paid_flag',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'emp_len',
 'loan_inc_ratio',
 'instal_inc_ratio',
#  'purpose-onehot',
#  'grade-onehot',
#  'sub_grade-onehot',
#  'verification_status-onehot',
#  'home_ownership-onehot'
)

In [194]:
loan_df_final.take(1)

[Row(paid_flag=0, loan_amnt=5000.0, funded_amnt=5000.0, funded_amnt_inv=4975.0, int_rate=10.649999618530273, installment=162.8699951171875, annual_inc=24000.0, dti=27.649999618530273, delinq_2yrs=0, inq_last_6mths=1, open_acc=3, pub_rec=0, revol_bal=13648.0, revol_util=83.69999694824219, total_acc=9, acc_now_delinq=0, tot_coll_amt=0.0, tot_cur_bal=0.0, total_rev_hi_lim=0, emp_len=10, loan_inc_ratio=0.2083333283662796, instal_inc_ratio=0.006786249577999115)]

In [178]:
# count how many nulls in each column
# def count_null(c):
#     return sum(col(c).isNull().cast("integer")).alias(c)

# exprs = [count_null(c) for c in loan_df9.columns]
# loan_df9.agg(*exprs).toPandas().T

## 4. Modeling

In [179]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

# Load and parse the data file into an RDD of LabeledPoint.
data = loan_df_final.rdd.map(lambda row: LabeledPoint(row[0], row[1:]))
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=11, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=15, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())
print('Test Error = ' + str(testErr))

Test Error = 0.181634808036
